# Entraînement du modèle YOLO
## Projet : Détection de conflits homme-faune

**Équipe DIA6 - Novembre 2024**

---

### Instructions d'utilisation

1. **Activer le GPU** : Runtime > Change runtime type > T4 GPU
2. **Exécuter les cellules dans l'ordre** (Shift + Enter)
3. **Durée totale** : ~2 heures (dont 1h30 d'entraînement automatique)
4. **Points de vérification** : Après chaque section, vérifier que tout est OK

---

### Prérequis

- Dataset uploadé sur Google Drive dans : `MyDrive/ProjetGreenAI/datasets/wildlife_human_final/`
- Connexion internet stable
- Compte Google

---

## Section 1 : Vérification de l'environnement

**Objectif** : Vérifier que le GPU est activé et que l'environnement est correct

**Résultat attendu** : Vous devez voir "GPU : Tesla T4" ou similaire

In [ ]:
import sys
import torch

print("Configuration de l'environnement")
print("-" * 50)
print(f"Python version : {sys.version}")
print(f"PyTorch version : {torch.__version__}")
print(f"CUDA disponible : {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU : {torch.cuda.get_device_name(0)}")
    print("\n✅ GPU detecte - Vous pouvez continuer")
else:
    print("\n❌ ATTENTION : GPU non detecte !")
    print("Allez dans Runtime > Change runtime type > Selectionnez GPU")
    
print("-" * 50)

## Section 2 : Installation des dépendances

**Objectif** : Installer Ultralytics (librairie YOLO)

**Durée** : 1-2 minutes

**Note** : Des messages d'avertissement peuvent apparaître, c'est normal

In [ ]:
print("Installation d'Ultralytics...")
!pip install ultralytics -q
print("\n✅ Installation terminee")

# Verification
import ultralytics
print(f"Ultralytics version : {ultralytics.__version__}")
ultralytics.checks()

## Section 3 : Connexion à Google Drive

**Objectif** : Monter Google Drive pour accéder au dataset

**Action requise** : 
1. Une fenêtre va s'ouvrir
2. Cliquez sur "Connect to Google Drive"
3. Sélectionnez votre compte Google
4. Cliquez sur "Allow"

**Résultat attendu** : "Drive monte avec succes" + "Dataset trouve"

In [ ]:
from google.colab import drive
import os

print("Connexion a Google Drive...")
drive.mount('/content/drive')
print("\n✅ Drive monte avec succes")

# Verifier l'acces au dataset
DATASET_PATH = '/content/drive/MyDrive/ProjetGreenAI/datasets/wildlife_human_final'

print(f"\nVerification du dataset...")
if os.path.exists(DATASET_PATH):
    print(f"✅ Dataset trouve")
    print(f"Emplacement : {DATASET_PATH}")
else:
    print(f"❌ ERREUR : Dataset non trouve")
    print(f"Emplacement attendu : {DATASET_PATH}")
    print("\nVerifiez :")
    print("1. Que vous avez bien uploade le dossier sur Google Drive")
    print("2. Que le chemin est correct")
    print("3. Que le dossier s'appelle exactement 'wildlife_human_final'")

## Section 4 : Vérification de la structure du dataset

**Objectif** : Vérifier que toutes les images et labels sont présents

**Résultat attendu** :
- train  : 1650 images, 1650 labels
- valid  :  205 images,  205 labels
- test   :  208 images,  208 labels

**Si vous voyez "DOSSIER MANQUANT", arrêtez-vous et vérifiez votre Google Drive**

In [ ]:
from pathlib import Path

print("\nStructure du dataset")
print("-" * 50)

all_ok = True

for split in ['train', 'valid', 'test']:
    images_dir = Path(DATASET_PATH) / split / 'images'
    labels_dir = Path(DATASET_PATH) / split / 'labels'
    
    if images_dir.exists() and labels_dir.exists():
        n_images = len(list(images_dir.glob('*')))
        n_labels = len(list(labels_dir.glob('*.txt')))
        print(f"✅ {split:6s} : {n_images:4d} images, {n_labels:4d} labels")
    else:
        print(f"❌ {split:6s} : DOSSIER MANQUANT")
        all_ok = False
        if not images_dir.exists():
            print(f"   Manque : {images_dir}")
        if not labels_dir.exists():
            print(f"   Manque : {labels_dir}")

print("-" * 50)

if all_ok:
    print("\n✅ Structure du dataset correcte - Vous pouvez continuer")
else:
    print("\n❌ Structure incorrecte - Verifiez votre Google Drive")

## Section 5 : Vérification du fichier data.yaml

**Objectif** : Vérifier que le fichier de configuration existe et est correct

**Contenu attendu** :
- path: chemin vers le dataset
- train, val, test: chemins relatifs
- names: liste des 5 classes

In [ ]:
YAML_PATH = Path(DATASET_PATH) / 'data.yaml'

if YAML_PATH.exists():
    print("\n✅ Fichier data.yaml trouve")
    print("\nContenu du fichier :")
    print("-" * 50)
    with open(YAML_PATH, 'r') as f:
        content = f.read()
        print(content)
    print("-" * 50)
else:
    print("\n❌ ERREUR : Fichier data.yaml non trouve")
    print(f"Emplacement attendu : {YAML_PATH}")
    print("\nCreation du fichier...")
    
    yaml_content = f"""path: {DATASET_PATH}
train: train/images
val: valid/images
test: test/images

names:
  0: buffalo
  1: elephant
  2: rhino
  3: zebra
  4: person

nc: 5
"""
    
    with open(YAML_PATH, 'w') as f:
        f.write(yaml_content)
    
    print("✅ Fichier data.yaml cree avec succes")

## Section 6 : Chargement du modèle YOLO

**Objectif** : Charger le modèle YOLOv8 nano pré-entraîné

**Note** : Le modèle (6 MB) se télécharge automatiquement

In [ ]:
from ultralytics import YOLO

print("\nChargement du modele YOLOv8 nano...")
model = YOLO('yolov8n.pt')
print("✅ Modele charge avec succes")
print(f"Taille du modele : ~6 MB")
print(f"Architecture : YOLOv8 nano")
print(f"Pre-entraine sur : COCO dataset (80 classes)")

---
## 🚀 Section 7 : ENTRAÎNEMENT DU MODÈLE

**⚠️ CETTE CELLULE PREND 1-2 HEURES ⚠️**

### Paramètres d'entraînement
- **Époques** : 50 (nombre de fois que le modèle voit toutes les images)
- **Batch size** : 16 (nombre d'images traitées simultanément)
- **Image size** : 640x640 pixels
- **Patience** : 10 (arrêt automatique si pas d'amélioration pendant 10 époques)

### Pendant l'entraînement
- Vous verrez défiler les époques (Epoch 1/50, 2/50, etc.)
- Des métriques s'afficheront (mAP, precision, recall)
- **NE FERMEZ PAS LE NAVIGATEUR**
- Vous pouvez minimiser l'onglet et faire autre chose

### Après l'entraînement
- Le meilleur modèle est sauvegardé automatiquement
- Des graphiques sont générés automatiquement

**Prêt ? Exécutez la cellule ci-dessous !**

---

In [ ]:
print("\n" + "="*50)
print("DEMARRAGE DE L'ENTRAINEMENT")
print("="*50)
print("Duree estimee : 1-2 heures")
print("NE FERMEZ PAS LE NAVIGATEUR")
print("="*50 + "\n")

# Parametres
EPOCHS = 50
BATCH_SIZE = 16
IMAGE_SIZE = 640

# Entrainement
results = model.train(
    data=str(YAML_PATH),
    epochs=EPOCHS,
    imgsz=IMAGE_SIZE,
    batch=BATCH_SIZE,
    patience=10,
    device=0,
    project='runs/detect',
    name='wildlife_conflict',
    exist_ok=True,
    verbose=True,
    plots=True,
    save=True
)

print("\n" + "="*50)
print("✅ ENTRAINEMENT TERMINE")
print("="*50)

## Section 8 : Résultats de l'entraînement

**Objectif** : Afficher les métriques de performance du modèle

**Métriques importantes** :
- **mAP50** : Précision moyenne à 50% de seuil (>0.85 = très bon)
- **Precision** : % de détections correctes
- **Recall** : % d'objets trouvés

In [ ]:
print("\nResultats de l'entrainement")
print("-" * 50)

# Metriques
metrics = model.metrics

print(f"mAP50     : {metrics.box.map50:.3f}")
print(f"mAP50-95  : {metrics.box.map:.3f}")
print(f"Precision : {metrics.box.mp:.3f}")
print(f"Recall    : {metrics.box.mr:.3f}")

print("-" * 50)

# Interpretation
if metrics.box.map50 > 0.85:
    print("\n✅ Excellent ! Le modele est tres performant")
elif metrics.box.map50 > 0.75:
    print("\n✅ Bon ! Le modele est performant")
elif metrics.box.map50 > 0.65:
    print("\n⚠️ Moyen - Le modele est correct mais peut etre ameliore")
else:
    print("\n❌ Faible - Le modele necessite plus d'entrainement ou plus de donnees")

## Section 9 : Visualisation des résultats

**Objectif** : Afficher les courbes d'entraînement et la matrice de confusion

**Graphiques générés** :
- Courbes d'entraînement (loss, mAP, etc.)
- Matrice de confusion (quelles classes sont confondues)
- Exemples de prédictions

In [ ]:
from IPython.display import Image, display

results_dir = Path('runs/detect/wildlife_conflict')

# Courbes d'entrainement
results_png = results_dir / 'results.png'
if results_png.exists():
    print("\nCourbes d'entrainement")
    print("-" * 50)
    display(Image(filename=str(results_png)))

# Matrice de confusion
confusion = results_dir / 'confusion_matrix.png'
if confusion.exists():
    print("\nMatrice de confusion")
    print("-" * 50)
    display(Image(filename=str(confusion)))

# Exemples de predictions
val_batch = results_dir / 'val_batch0_pred.jpg'
if val_batch.exists():
    print("\nExemples de predictions sur le validation set")
    print("-" * 50)
    display(Image(filename=str(val_batch)))

## Section 10 : Sauvegarde du modèle sur Google Drive

**Objectif** : Sauvegarder le meilleur modèle sur Google Drive pour le réutiliser

**Emplacement** : `MyDrive/ProjetGreenAI/models/wildlife_best.pt`

In [ ]:
import shutil

# Sauvegarder le modele sur Drive
best_model_path = 'runs/detect/wildlife_conflict/weights/best.pt'
save_dir = '/content/drive/MyDrive/ProjetGreenAI/models/'

os.makedirs(save_dir, exist_ok=True)
shutil.copy(best_model_path, os.path.join(save_dir, 'wildlife_best.pt'))

print("✅ Modele sauvegarde sur Google Drive")
print(f"Emplacement : {save_dir}wildlife_best.pt")
print("\nVous pouvez maintenant fermer ce notebook")
print("Le modele est pret a etre utilise !")

---
## 📋 Résumé et prochaines étapes

### Ce qui a été fait
✅ Environnement vérifié  
✅ Dataset chargé et vérifié  
✅ Modèle YOLO entraîné pendant 50 époques  
✅ Résultats générés et visualisés  
✅ Modèle sauvegardé sur Google Drive  

### Fichiers générés
- **Modèle entraîné** : `MyDrive/ProjetGreenAI/models/wildlife_best.pt`
- **Résultats** : `runs/detect/wildlife_conflict/`
- **Graphiques** : Dans le dossier results

### Prochaines étapes
1. Tester le modèle sur les 40 images réelles
2. Créer les cartes de densité spatiale
3. Identifier les zones de conflit
4. Préparer la présentation

### Ressources
- Documentation YOLO : https://docs.ultralytics.com
- Repository GitHub : https://github.com/luna-eng-data/wildlife-conflict-detection

---